##  Product_Data

In [2]:
import os
import pandas as pd
import numpy as np
from IPython.display import clear_output # Count iteration times
# from datetime import datetime # Record time

In [3]:
df = pd.read_csv('../data/processed/user_data.csv')

products = pd.read_csv('../data/raw/products.csv')
product_to_aisle = dict(zip(products['product_id'], products['aisle_id']))
product_to_department = dict(zip(products['product_id'], products['department_id']))
product_to_name = dict(zip(products['product_id'], products['product_name']))

user_ids = []
product_ids = []
aisle_ids = []
department_ids = []
product_names = []
eval_sets = []

is_ordered_histories = []
index_in_order_histories = []
order_size_histories = []
reorder_size_histories = []
order_dow_histories = []
order_hour_histories = []
days_since_prior_order_histories = []
order_number_histories = []

labels = []

longest = 0
    

In [4]:
longest = 0
for _, row in df.iterrows():
    if _ % 10000 == 0:
        print(_)
        data = [
        user_ids,
        product_ids,
        aisle_ids,
        department_ids,
        product_names,
        is_ordered_histories,
        index_in_order_histories,
        order_size_histories,
        reorder_size_histories,
        order_dow_histories,
        order_hour_histories,
        days_since_prior_order_histories,
        order_number_histories,
        labels,
        eval_sets
        ]
        # Length Check
        print(list(map(len, data)))

    user_id = row['user_id']
    eval_set = row['eval_set']
    products = row['product_ids']

    products, next_products = ' '.join(products.split()[:-1]), products.split()[-1]

    reorders = row['reorders']
    reorders, next_reorders = ' '.join(reorders.split()[:-1]), reorders.split()[-1]

    product_set = set([int(j) for i in products.split() for j in i.split('_')])
    next_product_set = set([int(i) for i in next_products.split('_')])

    orders = [map(int, i.split('_')) for i in products.split()]
    reorders = [map(int, i.split('_')) for i in reorders.split()]
    next_reorders = map(int, next_reorders.split('_'))

    for product_id in product_set:

        user_ids.append(user_id)
        product_ids.append(product_id)
        labels.append(int(product_id in next_product_set) if eval_set == 'train' else -1)
        eval_sets.append(eval_set) # Newly added

        ### Handle Null
        if product_id in product_to_aisle:
            aisle_ids.append(product_to_aisle[product_id])
        else:
            aisle_ids.append('0')  # or some other default value

        if product_id in product_to_department:
            department_ids.append(product_to_department[product_id])
        else:
            department_ids.append('0')  # or some other default value

        if product_id in product_to_name:
            product_names.append(product_to_name[product_id])
        else:
            product_names.append('0')  # or some other default value

        is_ordered = []
        index_in_order = []
        order_size = []
        reorder_size = []

        prior_products = set()
        for order in orders:
            is_ordered.append(str(int(product_id in order)))
            order_list = list(order)
            index_in_order.append(str(order_list.index(product_id) + 1) if product_id in order_list else '0')
            order_size.append(str(len(list(order))))
            reorder_size.append(str(len(list(prior_products & set(order)))))
            prior_products |= set(order)

        is_ordered = ' '.join(is_ordered)
        index_in_order = ' '.join(index_in_order)
        order_size = ' '.join(order_size)
        reorder_size = ' '.join(reorder_size)

        is_ordered_histories.append(is_ordered)
        index_in_order_histories.append(index_in_order)
        order_size_histories.append(order_size)
        reorder_size_histories.append(reorder_size)
        order_dow_histories.append(row['order_dows'])
        order_hour_histories.append(row['order_hours'])
        days_since_prior_order_histories.append(row['days_since_prior_orders'])
        order_number_histories.append(row['order_numbers'])

    user_ids.append(user_id)
    product_ids.append(0)
    # labels.append(int(max(next_reorders) == 0) if eval_set == 'train' else -1)
    # Do not consider the case "no order"
    labels.append(-1)

    aisle_ids.append(0)
    department_ids.append(0)
    product_names.append(0)
    eval_sets.append(eval_set)

    is_ordered = []
    index_in_order = []
    order_size = []
    reorder_size = []

    for reorder in reorders:
        is_ordered.append(str(int(max(reorder) == 0)))
        index_in_order.append(str(0))
        order_size.append(str(len(list(reorder))))
        reorder_size.append(str(sum(reorder)))

    is_ordered = ' '.join(is_ordered)
    index_in_order = ' '.join(index_in_order)
    order_size = ' '.join(order_size)
    reorder_size = ' '.join(reorder_size)

    is_ordered_histories.append(is_ordered)
    index_in_order_histories.append(index_in_order)
    order_size_histories.append(order_size)
    reorder_size_histories.append(reorder_size)
    order_dow_histories.append(row['order_dows'])
    order_hour_histories.append(row['order_hours'])
    days_since_prior_order_histories.append(row['days_since_prior_orders'])
    order_number_histories.append(row['order_numbers'])

0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10000
[171667, 171667, 171667, 171667, 171667, 171667, 171667, 171667, 171667, 171667, 171667, 171667, 171667, 171667, 171667]
20000
[215196, 215196, 215196, 215196, 215196, 215196, 215196, 215196, 215196, 215196, 215196, 215196, 215196, 215196, 215196]
30000
[254008, 254008, 254008, 254008, 254008, 254008, 254008, 254008, 254008, 254008, 254008, 254008, 254008, 254008, 254008]
40000
[288774, 288774, 288774, 288774, 288774, 288774, 288774, 288774, 288774, 288774, 288774, 288774, 288774, 288774, 288774]
50000
[319833, 319833, 319833, 319833, 319833, 319833, 319833, 319833, 319833, 319833, 319833, 319833, 319833, 319833, 319833]
60000
[347354, 347354, 347354, 347354, 347354, 347354, 347354, 347354, 347354, 347354, 347354, 347354, 347354, 347354, 347354]
70000
[371636, 371636, 371636, 371636, 371636, 371636, 371636, 371636, 371636, 371636, 371636, 371636, 371636, 371636, 371636]
80000
[393054, 393054, 393054, 393054, 393054, 393054, 393054, 

In [5]:
data = [
        user_ids,
        product_ids,
        aisle_ids,
        department_ids,
        product_names,
        is_ordered_histories,
        index_in_order_histories,
        order_size_histories,
        reorder_size_histories,
        order_dow_histories,
        order_hour_histories,
        days_since_prior_order_histories,
        order_number_histories,
        labels,
        eval_sets
    ]
# Length Check

list(map(len, data))

[458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967,
 458967]

In [6]:
columns = [
    'user_id',
    'product_id',
    'aisle_id',
    'department_id',
    'product_name',
    'is_ordered_history',
    'index_in_order_history',
    'order_size_history',
    'reorder_size_history',
    'order_dow_history',
    'order_hour_history',
    'days_since_prior_order_history',
    'order_number_history',
    'label',
    'eval_set'
]
if not os.path.isdir('../data/processed'):
    os.makedirs('../data/processed')

df = pd.DataFrame(dict(zip(columns, data)))
df.to_csv('../data/processed/product_data.csv', index=False)
